<a href="https://colab.research.google.com/github/devpandey2010/BAGGING-RANDOM-fOREST-/blob/main/OOB_score_and_Multimodel_regresor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
x,y=make_classification(n_samples=1000,n_features=20,n_classes=2,random_state=1)

In [ ]:
rf=RandomForestClassifier(n_estimators=100,oob_score=True,random_state=1)

In [ ]:
rf.fit(x,y)

RandomForestClassifier(oob_score=True, random_state=1)

In [ ]:
rf.oob_score_

0.861

In [ ]:
df=sns.load_dataset("tips")

In [ ]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [ ]:
x1=df.iloc[:,1:]
y1=df.iloc[:,0:1]

In [ ]:
x1

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,5.92,Male,No,Sat,Dinner,3
240,2.00,Female,Yes,Sat,Dinner,2
241,2.00,Male,Yes,Sat,Dinner,2
242,1.75,Male,No,Sat,Dinner,2


In [ ]:
y1

,total_bill
0,16.99
1,10.34
2,21.01
3,23.68
4,24.59
...,...
239,29.03
240,27.18
241,22.67
242,17.82


In [ ]:
#predict toatl bill
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [ ]:
#automatic feature enggennering
from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test=train_test_split(x1,y1,test_size=0.2,random_state=1)
x1_train.shape,x1_test.shape


((195, 6), (49, 6))

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
numeric_cols=["tip","size"]
cat_cols=["sex","smoker","day","time"]


In [ ]:
num_pipeline=Pipeline(steps=[("impuatation",SimpleImputer(strategy="median")),("scaling",StandardScaler())])
cat_pipeline=Pipeline(steps=[("impuatation",SimpleImputer(strategy="most_frequent")),("encoding",OneHotEncoder())])

In [ ]:
preprocessor=ColumnTransformer([("numeric_cols",num_pipeline,numeric_cols),("cat_cols",cat_pipeline,cat_cols)])

In [ ]:
x1_train=preprocessor.fit_transform(x1_train)
x1_test=preprocessor.transform(x1_test)
x1_train,x1_test

(array([[-1.47443803, -0.57766863,  1.        , ...,  0.        ,
          1.        ,  0.        ],
        [-0.71612531,  1.47042924,  0.        , ...,  0.        ,
          1.        ,  0.        ],
        [ 1.19880579,  1.47042924,  0.        , ...,  0.        ,
          1.        ,  0.        ],
        ...,
        [ 0.43283335, -0.57766863,  1.        , ...,  1.        ,
          0.        ,  1.        ],
        [-1.29060464, -0.57766863,  0.        , ...,  0.        ,
          1.        ,  0.        ],
        [ 0.1034652 ,  0.44638031,  1.        , ...,  0.        ,
          1.        ,  0.        ]]),
 array([[-1.48209775, -1.60171757,  1.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ,  0.        ],
        [ 0.04984713, -0.57766863,  1.        ,  0.        ,  1.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
          1.        ,  0.        ],
     

In [ ]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
models={"svm":SVR(),
       "Dt":DecisionTreeRegressor(),
       "regression":LinearRegression(),
       "random forest":RandomForestRegressor()}
models


{'svm': SVR(),
 'Dt': DecisionTreeRegressor(),
 'regression': LinearRegression(),
 'random forest': RandomForestRegressor()}

In [ ]:
from sklearn.metrics import r2_score
def model_train_eval(x1_train,x1_test,y1_train,y1_test,models):
  evaluation={}
  for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(x1_train,y1_train)
    y1_pred=model.predict(x1_test)
    model_score=r2_score(y1_test,y1_pred)
    evaluation[list(models.keys())[i]]=model_score
  return evaluation



In [ ]:
model_train_eval(x1_train,x1_test,y1_train,y1_test,models)

{'svm': 0.4069433055293954,
 'Dt': 0.14724348667385,
 'regression': 0.6719006761625261,
 'random forest': 0.6435107470379693}